### Getting Started With TensorFlow
### https://www.tensorflow.org/get_started/get_started

# Tensors

3 # a rank 0 tensor; this is a scalar with shape []

[1., 2., 3.] # a rank 1 tensor; this is a vector with shape [3]

[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]

[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]

## TensorFlow Core tutorial

In [1]:
import tensorflow as tf

In [2]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0, dtype=tf.float32)
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [3]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [4]:
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ", sess.run(node3))

node3:  Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3):  7.0


In [5]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # or tf.add(a, b)

In [7]:
print(sess.run(adder_node, feed_dict={a: 3, b:4}))
print(sess.run(adder_node, feed_dict={a: [3, 4.5], b:[4.1, 6]}))

7.0
[  7.0999999  10.5      ]


In [8]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 4, b:5}))

27.0


In [9]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(dtype=tf.float32)

linear_model = W * x + b

In [11]:
init = tf.global_variables_initializer()
sess.run(init)

In [12]:
print(sess.run(linear_model, {x: [1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [13]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)

print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


In [14]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])

print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


## tf.train API

In [16]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [18]:
sess.run(init)
for i in range(1000):
    sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
    
print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


### Complete Program

In [19]:
import tensorflow as tf

# Model Parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)

# Model input and output
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

# Model
linear_model = W * x + b

# loss
loss = tf.reduce_sum(tf.square(linear_model - y))

# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

# training loop
init = tf.global_variables_initializer()
sess = tf.Session()

sess.run(init)

for i in range(1000):
    sess.run(train, {x: x_train, y: y_train})
    
# evalute training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s" % (curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


## tf.estimator

tf.estimator is a high-level TensorFlow library that simplifies the mechanics of machine learning, including the following:

- running training loops
- running evaluation loops
- managing data sets

tf.estimator defines many common models.

### Basic Usage

In [26]:
import tensorflow as tf
import numpy as np

feature_columns = [tf.feature_column.numeric_column("x", shape = [1])]

estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])

input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

estimator.train(input_fn=input_fn, steps=1000)

train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r", train_metrics)
print("eval metrics: %r", eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/ss/z_kczhtn2s18h3crds_dlgpn77x_9l/T/tmpepssoh5o', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/ss/z_kczhtn2s18h3crds_dlgpn77x_9l/T/tmpepssoh5o/model.ckpt.
INFO:tensorflow:loss = 15.0, step = 1
INFO:tensorflow:global_step/sec: 1081.39
INFO:tensorflow:loss = 0.239676, step = 101 (0.094 sec)
INFO:tensorflow:global_step/sec: 1111.01
INFO:tensorflow:loss = 0.0483582, step = 201 (0.090 sec)
INFO:tensorflow:global_step/sec: 1270.28
INFO:tensorflow:loss = 0.0068667, step = 301 (0.079 sec)
INFO:tensorflow:global_step/sec: 1105.72
INFO:tensorflow:loss = 0.0014456, step = 401 (0.091 sec)
INFO:tensorflow:global_s

### A custom model

In [27]:
import numpy as np
import tensorflow as tf

# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W * features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # EstimatorSpec connects subgraphs we built to the
  # appropriate functionality.
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/ss/z_kczhtn2s18h3crds_dlgpn77x_9l/T/tmpotgifpkh', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/ss/z_kczhtn2s18h3crds_dlgpn77x_9l/T/tmpotgifpkh/model.ckpt.
INFO:tensorflow:loss = 17.3073748566, step = 1
INFO:tensorflow:global_step/sec: 1381.08
INFO:tensorflow:loss = 0.0314294758896, step = 101 (0.075 sec)
INFO:tensorflow:global_step/sec: 1267.3
INFO:tensorflow:loss = 0.00522201149971, step = 201 (0.078 sec)
INFO:tensorflow:global_step/sec: 1210.41
INFO:tensorflow:loss = 0.000360732436226, step = 301 (0.083 sec)
INFO:tensorflow:global_step/sec: 1342.42
INFO:tensorflow:loss = 5.11407158179e-06, step = 40